In [3]:

from langchain_community.embeddings import LlamaCppEmbeddings, HuggingFaceEmbeddings, OpenAIEmbeddings
from langchain_pinecone import PineconeVectorStore
import os
from langchain_core.callbacks import CallbackManager, StreamingStdOutCallbackHandler
from langchain_community.llms import LlamaCpp
from pinecone import Pinecone, ServerlessSpec

from langchain_pinecone import PineconeVectorStore
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.document_loaders import TextLoader
from dotenv import load_dotenv

from langchain_community.llms import Ollama


d:\DigiMark\Chat-with-Doc-LLAMA-Langcahin\.venv\lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [2]:
load_dotenv() # Loading Enviroment Variables
pc = Pinecone(api_key=os.environ.get('PINE_CONE_API_KEY')) # Setting Pine Cone API Key



In [3]:
loader = TextLoader(r"D:\DigiMark\Chat-with-Doc-LLAMA-Langcahin\data\monkey1.txt")
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

In [4]:
ollama_emb = OllamaEmbeddings(
    model="llama3",
    num_gpu=0,
    num_ctx=0,
    base_url='http://127.0.0.1:11434'
)

In [6]:
help(OllamaEmbeddings)

Help on class OllamaEmbeddings in module langchain_community.embeddings.ollama:

class OllamaEmbeddings(pydantic.v1.main.BaseModel, langchain_core.embeddings.embeddings.Embeddings)
 |  OllamaEmbeddings(*, base_url: str = 'http://localhost:11434', model: str = 'llama2', embed_instruction: str = 'passage: ', query_instruction: str = 'query: ', mirostat: Optional[int] = None, mirostat_eta: Optional[float] = None, mirostat_tau: Optional[float] = None, num_ctx: Optional[int] = None, num_gpu: Optional[int] = None, num_thread: Optional[int] = None, repeat_last_n: Optional[int] = None, repeat_penalty: Optional[float] = None, temperature: Optional[float] = None, stop: Optional[List[str]] = None, tfs_z: Optional[float] = None, top_k: Optional[int] = None, top_p: Optional[float] = None, show_progress: bool = False, headers: Optional[dict] = None, model_kwargs: Optional[dict] = None) -> None
 |  
 |  Ollama locally runs large language models.
 |  
 |  To use, follow the instructions at https://oll

In [ ]:
'http://127.0.0.1:11434'

In [5]:
ollama_emb.embed_query("apple")

[-0.7210520505905151,
 -2.7212562561035156,
 2.5227110385894775,
 -2.0779500007629395,
 1.3411064147949219,
 2.226654052734375,
 1.8355079889297485,
 1.369401216506958,
 -3.7224197387695312,
 -1.9915339946746826,
 1.7781661748886108,
 -0.16624009609222412,
 0.5139321684837341,
 2.93908429145813,
 1.2529476881027222,
 3.2869961261749268,
 -0.9226197004318237,
 -1.03190016746521,
 -2.4092774391174316,
 1.95339035987854,
 -2.6631057262420654,
 -1.0131028890609741,
 2.455963134765625,
 -1.583998680114746,
 -1.8110244274139404,
 0.8823566436767578,
 0.23536871373653412,
 4.715279579162598,
 1.6706933975219727,
 0.8011358380317688,
 0.7992376685142517,
 4.382404327392578,
 0.6707046031951904,
 2.0226798057556152,
 -0.03738769143819809,
 -1.4566441774368286,
 -1.6596076488494873,
 1.3750149011611938,
 0.1689748913049698,
 -0.0864853784441948,
 -0.4330006241798401,
 1.0275887250900269,
 2.028829336166382,
 -1.8705939054489136,
 2.9674479961395264,
 1.218674659729004,
 0.22435033321380615,
 -0.

In [ ]:
PineconeVectorStore.from_documents(docs,embedding=ollama_emb,index_name = 'ollama')

In [4]:
help(Ollama)

Help on class Ollama in module langchain_community.llms.ollama:

class Ollama(langchain_core.language_models.llms.BaseLLM, _OllamaCommon)
 |  Ollama(*, name: Optional[str] = None, cache: ForwardRef('Union[BaseCache, bool, None]') = None, verbose: bool = None, callbacks: ForwardRef('Callbacks') = None, tags: Optional[List[str]] = None, metadata: Optional[Dict[str, Any]] = None, custom_get_token_ids: Optional[Callable[[str], List[int]]] = None, base_url: str = 'http://localhost:11434', model: str = 'llama2', mirostat: Optional[int] = None, mirostat_eta: Optional[float] = None, mirostat_tau: Optional[float] = None, num_ctx: Optional[int] = None, num_gpu: Optional[int] = None, num_thread: Optional[int] = None, num_predict: Optional[int] = None, repeat_last_n: Optional[int] = None, repeat_penalty: Optional[float] = None, temperature: Optional[float] = None, stop: Optional[List[str]] = None, tfs_z: Optional[float] = None, top_k: Optional[int] = None, top_p: Optional[float] = None, system: Op

In [5]:
llm = Ollama(model="llama3",num_gpu=1,num_ctx=1536, base_url="http://127.0.0.1:11434")


In [8]:
help(llm.invoke)

Help on method invoke in module langchain_core.language_models.llms:

invoke(input: 'LanguageModelInput', config: 'Optional[RunnableConfig]' = None, *, stop: 'Optional[List[str]]' = None, **kwargs: 'Any') -> 'str' method of langchain_community.llms.ollama.Ollama instance
    Transform a single input into an output. Override to implement.
    
    Args:
        input: The input to the runnable.
        config: A config to use when invoking the runnable.
           The config supports standard keys like 'tags', 'metadata' for tracing
           purposes, 'max_concurrency' for controlling how much work to do
           in parallel, and other keys. Please refer to the RunnableConfig
           for more details.
    
    Returns:
        The output of the runnable.



In [6]:
resp = llm.invoke("Who is an apple? the answer should be in less than 50 words")

In [7]:
resp

'A simple one! An Apple is a technology company that designs, manufactures, and markets consumer electronics, computer software, and online services. They are known for their iPhones, Mac computers, iPads, and other innovative products. Founded by Steve Jobs and Steve Wozniak in 1976.'

In [ ]:
pc_v =  PineconeVectorStore(index_name='ollama',pinecone_api_key=os.environ.get('PINECONE_API_KEY'),embedding=ollama_emb)
retriever=pc_v.as_retriever()


In [ ]:
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory



### Contextualize question ###
contextualize_q_system_prompt = """Given a chat history and the latest user question which might reference context in the chat history, formulate a standalone question which can be understood without the chat history. Do NOT answer the question, just reformulate it if needed and otherwise return it as is."""
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)


### Answer question ###
qa_system_prompt = """You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise
{context}"""
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", qa_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)


### Statefully manage chat history ###
store = {}


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
)

In [ ]:
question = 'give me a summary of the story?'

In [ ]:
conversational_rag_chain.invoke({"input": question}, {'configurable': {'session_id': '2'}})

In [ ]:
question = 'tell me about them both that is monkeys.'
conversational_rag_chain.invoke({"input": question,}, {'configurable': {'session_id': '1'}})

In [ ]:
help(OllamaEmbeddings)